## 02 - The True Power of Qurry

In the previous section, we have learned how to use the basic function of Qurry. In this section, we will learn that Qurry provides a more advanced approach, where the true power of Qurry lies.



In [1]:
from qurry import EntropyMeasure, BackendManager
from qiskit import (
    IBMQ, execute, transpile,
    QuantumRegister, ClassicalRegister, QuantumCircuit,
)
from pathlib import Path

experiment_executor_02 = EntropyMeasure()

## 2.1 - `BackendManager`, the advanced version `backendWrapper`

In the previous chapter, we used `backendWrapper` to import the backend, and select a provider as a input to load IBM Backend from the IBM Account. 
Now, `BackendManager` will load accout automatically, and you can use `BackendManager` to load backend directly.

```python
BackendManager.save_account('your_token')
```
To save your account, if you have already saved, you can ignore this step.

```python
backend = BackendManager(instance='ibm-q/open/main')
```
or 
```python
backend = BackendManager(hub='ibm-q', group='open', project='main')
```


In [3]:
# BackendManager.save_account('your_token')
backend = BackendManager(hub='ibm-q', group='open', project='main')


| Provider by 'qiskit_ibm_provider'.


You can just type the name of backend without the prefix of `'ibmq_'` or `'ibm_'`.


In [4]:
backend('ibm_sherbrooke'), backend('sherbrooke')

(<IBMBackend('ibm_sherbrooke')>, <IBMBackend('ibm_sherbrooke')>)

## 2.2 - Launching a multiJob

Consider a scenario where you have multiple circuits that you want to run on a backend at the same time, you can use `multiOutput` to launch a multiJob.

For the example below, we will show how to submit 100 circuits with 100 sets of randomized unitaries and 4096 shots, and mesure the entropy with multiple subsystem divisions at the same time.

### 1. Loading 100 circuits

(Topological Paramagnetic State is already the most complicated case in Qurry, so we will use it as our example. Ususally I prefer some more complicated circuits like a spin chain model with 20+ trotter steps, but it's not necessary for this example.)



In [4]:
from qurry.recipe import TopologicalParamagnet
sample = TopologicalParamagnet(8, 'period')
print(sample)

     ┌───┐         
q_0: ┤ H ├─■─────■─
     ├───┤ │     │ 
q_1: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_2: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_3: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_4: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_5: ┤ H ├─■──■──┼─
     ├───┤    │  │ 
q_6: ┤ H ├─■──■──┼─
     ├───┤ │     │ 
q_7: ┤ H ├─■─────■─
     └───┘         


Loading circuits to `Qurry`

In [5]:
for i in range(100):
    experiment_executor_02.add(sample)


Check the name or serial number of the circuit in `Qurry` by `experiment_executor_02.waves.keys()`


In [6]:
print(experiment_executor_02.waves.keys())

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99])


### 2. Preparing configurations

To perform randomized measurement, multiple randomized unitary configurations required. We use a list to store all configurations. 

For randomized measurement, each configuration is in this form of dictionary:

```python
{
    'waves': 'the circuit to measure',
    'tags': 'tags for this job',
    'times': 100,
}
```

In [7]:
from typing import TypedDict, Hashable, Iterable, Union, Optional

class randomizedConfig(TypedDict):
    wave: Hashable
    """The name or serial number of circuit in Qurry."""
    tags: Hashable
    """You can metion tags to filter experiment."""
    times: int
    """Default: 100 in :cls:`RandomizedMeasure`"""
    

In [8]:
config_list = [{
    'wave': i,
    'tags': ('topParamagnet', int(i/10)),
    'times': 100,
} for i in range(100)]

### 3. Launching multiJob

Attenetion, this example may make your computer go brrrr.

In [2]:
# please un-comment this cell to run it.

# hashID = experiment_executor_02.multiOutput(
#     config_list=config_list,
#     backend=backend('aer'),
#     save_location=Path('./'),
#     summoner_name='example.multiOutput',
#     shots=4096,
# )

# It takes 10m 41.2s to run this cell on my computer.

The `multiOutput` will return an ID of this multiJob, 

you can use it to check the status of this multiJob in `experiment_executor_02.multimanagers[(the ID returned)]`.

In [10]:
hashID # The hashID of this multiOutput

'd10186c6-debb-4685-9621-061b8d2a0577'

In [11]:
experiment_executor_02.multimanagers

{'d10186c6-debb-4685-9621-061b8d2a0577': <qurry.qurrium.multimanager.multimanager.MultiManager at 0x76afa00d3e90>}

### 4. The automatic export of multiJob

After the multiJob is finished, you may notice that there are some new files in your folder, they are the results of your multiJob.

Qurry will automatically export the results of multiJob to your folder, and you can use `multiRead` to load them. We will show how to use `multiRead` in later section.

### 5. Make multiple analysis with multiple subsystem divisions

Attenetion, this example may make your computer go brrrr, too.
The post-processing calculation already boosts by multiprocesses, cython and Rust, speeding up the calculation. In early version, such calculation can take all day, even few days to finish.

In [12]:
subsytems = [
    2, 4, 6, (2, 4), (4, 6), (2, 6), 
    (-2, 2), (-4, 2), (-2, 4), 3, (5, 7), (4, 7)]
# this is a list of subsystems that we want to measure

In [14]:
for d in subsytems:
    name = (
        f'N_A={"to".join([str(dd) for dd in d])}' if isinstance(d, tuple) 
        else f'N_A={d}')
    print("| Name:", name)
    
    # please decoment this cell to run it.
    
    # experiment_executor_02.multiAnalysis(
    #     summoner_id=hashID,
    #     degree=d,
    #     analysis_name=name,
    #     no_serialize=True,
    # )
    
# 38m 15.1s to run this cell on my computer.

| Name: N_A=2


| 0/100 - Analysis:  - 00:00 < ?

| "N_A=2" has been completed.
| Export multimanager...


| 0/9 - exporting - 00:00 < ?

| 0/2 - exporting quantity - 00:00 < ?

...

| "N_A=4to7" has been completed.
| Export multimanager...


| 0/9 - exporting - 00:00 < ?

| 0/13 - exporting quantity - 00:00 < ?

| Export multi.config.json for d10186c6-debb-4685-9621-061b8d2a0577


| 0/100 - Exporting and writring... - 00:00 < ?

## 2.3 Reading and Writing Data

In this section, we will show how to use `multiRead` to load the results of multiJob, and how to use `multiWrite` to save the results of multiJob.

In [20]:
hashID = experiment_executor_02.multiRead(
    save_location=Path('./'),
    summoner_name='example.multiOutput.qurry.002',
)

| MultiRead running...
| Retrieve example.multiOutput.qurry.002...
| at: example.multiOutput.qurry.002
| Found the tarfile 'example.multiOutput.qurry.002.qurry.tar.xz' in '.', decompressing is available.
| No multi.config file found, decompressing all files in the tarfile 'example.multiOutput.qurry.002.qurry.tar.xz'.
The following files '['example.multiOutput.qurry.002/N_A=2to6.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.002/N_A=2to6.quantity.json'.
The following files '['example.multiOutput.qurry.002/N_A=4to7.quantity.json']' are fitting giving 'taglist_name' and 'name', choosing the 'example.multiOutput.qurry.002/N_A=4to7.quantity.json'.


| 0/100   0%|          | - 100 experiments found, loading by 16 workers. - 00:00 < ?

## 2.4 Export and Compress

In this section, we will show how to use `multiWrite` to export the results of multiJob to a folder or export as a tar.xz file when `compress=True`.

In [17]:
experiment_executor_02.multiWrite(
    save_location=Path('./'),
    summoner_id=hashID,
    compress=True,
)

| Export multimanager...


| 0/9 - exporting - 00:00 < ?

| 0/13 - exporting quantity - 00:00 < ?

| Export multi.config.json for d10186c6-debb-4685-9621-061b8d2a0577


| 0/100 - Exporting and writring... - 00:00 < ?

| Compress multimanager of 'example.multiOutput.qurry.002'...done


'd10186c6-debb-4685-9621-061b8d2a0577'

---

## Post-Process Availablities and Version Info



In [18]:
from qurry import __version__
from qurry.process import AVAIBILITY_STATESHEET
print("| Qurry version:", __version__)
print(AVAIBILITY_STATESHEET)

| Qurry version: 0.7.1.dev3
--------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability           Python Cython Rust  
 - randomized_measure
   - entangled_core ............... True   True   True  
   - purity_cell .................. True   True   True  
   - wavefunction_overlap ......... True   True   None  
   - echo_cell .................... True   True   None  
 - utils
   - construct .................... True   True   True  
   - construct .................... True   None   True  
--------------------------------------------------------
   + True ..... Working normally.
   + False .... Exception occurred.
   + None ..... Not supported yet.
--------------------------------------------------------

